In [1]:
import pandas as pd
import ibm_db
import logging
from sqlalchemy import create_engine
import os
import glob
import datetime 


conn = ibm_db.connect("DATABASE=AYB_APPL;HOSTNAME=10.143.16.244;PORT=50000;PROTOCOL=TCPIP;UID=IOT_DATA;PWD=asd23fgh;", "", "")
if conn:
    print("Connected to the database")

Connected to the database


In [20]:
df=pd.read_csv(r"C:\Users\00220401626\Desktop\FMS\CsvData\2023\20231021.csv", encoding='cp932',dtype= object)
df['Timestamp']=pd.to_datetime(df['Timestamp'])
df

,Location,Timestamp,1.0μｍ,3.0μｍ,5.0μｍ,10.0μｍ,15.0μｍ,20.0μｍ,25.0μｍ,50.0μｍ
0,RP07,2023-10-21 00:01:23,0,0,0,0,0,0,0,0
1,RP09,2023-10-21 00:01:23,0,0,0,0,0,0,0,0
2,RP10,2023-10-21 00:01:22,14,3,0,0,0,0,0,0
3,RP04,2023-10-21 00:01:03,1,0,0,0,0,0,0,0
4,RP07,2023-10-21 00:02:53,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
12475,RP11,2023-10-21 23:59:53,0,0,0,0,0,0,0,0
12476,RP12,2023-10-21 23:56:14,0,0,0,0,0,0,0,0
12477,RP02,2023-10-21 23:59:32,1,0,0,0,0,0,0,0
12478,RP12,2023-10-21 23:57:44,0,0,0,0,0,0,0,0


In [21]:
insert_query = "INSERT INTO LIQUID_PARTICLE (LOCATION, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0,VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
stmt = ibm_db.prepare(conn, insert_query)
for index, row in df.iterrows():
    location = row['Location']
    timestamp = row['Timestamp']
    c1 = row['1.0μｍ']
    c3 = row['3.0μｍ']
    c5 = row['5.0μｍ']
    c10 = row['10.0μｍ']
    c15 = row['15.0μｍ']
    c20 = row['20.0μｍ']
    c25 = row['25.0μｍ']
    c50 = row['50.0μｍ']

    if ibm_db.execute(stmt, (location, timestamp, c1, c3, c5, c10, c15, c20, c25, c50)):
        print(f"{index + 1} row inserted")
    else:
        print(f"Error inserting row {index + 1}: {ibm_db.stmt_errormsg()}")
print('All Rows are successfully inserted')

1 row inserted
2 row inserted
3 row inserted
4 row inserted
5 row inserted
6 row inserted
7 row inserted
8 row inserted
9 row inserted
10 row inserted
11 row inserted
12 row inserted
13 row inserted
14 row inserted
15 row inserted
16 row inserted
17 row inserted
18 row inserted
19 row inserted
20 row inserted
21 row inserted
22 row inserted
23 row inserted
24 row inserted
25 row inserted
26 row inserted
27 row inserted
28 row inserted
29 row inserted
30 row inserted
31 row inserted
32 row inserted
33 row inserted
34 row inserted
35 row inserted
36 row inserted
37 row inserted
38 row inserted
39 row inserted
40 row inserted
41 row inserted
42 row inserted
43 row inserted
44 row inserted
45 row inserted
46 row inserted
47 row inserted
48 row inserted
49 row inserted
50 row inserted
51 row inserted
52 row inserted
53 row inserted
54 row inserted
55 row inserted
56 row inserted
57 row inserted
58 row inserted
59 row inserted
60 row inserted
61 row inserted
62 row inserted
63 row inserted
6

In [22]:
sql_query = "SELECT * FROM LIQUID_PARTICLE"
stmt = ibm_db.exec_immediate(conn, sql_query)

data = []
row = ibm_db.fetch_assoc(stmt)
while row:
    data.append(row)
    row = ibm_db.fetch_assoc(stmt)
df1 = pd.DataFrame(data)

In [24]:
# Hamma CSV Filelarni bitta dataframega joylash

path =  r"C:\Users\00220401626\Desktop\FMS\CsvData\2024"
csv_files = glob.glob(os.path.join(path, '*.csv'))

data_frames = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file, encoding='cp932',dtype= object)
    data_frames.append(df)
    
df2 = pd.concat(data_frames, ignore_index=True)
df2['Timestamp'] = pd.to_datetime(df2['Timestamp'])

In [30]:
filtered_data = df2[df2['Timestamp'] > df1['TS_DT'].max()]

In [33]:
insert_query = "INSERT INTO LIQUID_PARTICLE (LOCATION, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0,VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
stmt = ibm_db.prepare(conn, insert_query)
for index, row in  filtered_data.iterrows():
    location = row['Location']
    timestamp = row['Timestamp']
    c1 = row['1.0μｍ']
    c3 = row['3.0μｍ']
    c5 = row['5.0μｍ']
    c10 = row['10.0μｍ']
    c15 = row['15.0μｍ']
    c20 = row['20.0μｍ']
    c25 = row['25.0μｍ']
    c50 = row['50.0μｍ']

    if ibm_db.execute(stmt, (location, timestamp, c1, c3, c5, c10, c15, c20, c25, c50)):
        print(f"{index + 1} row inserted")
    else:
        print(f"Error inserting row {index + 1}: {ibm_db.stmt_errormsg()}")
print('All Rows are successfully inserted')

24961 row inserted
24962 row inserted
24963 row inserted
24964 row inserted
24965 row inserted
24966 row inserted
24967 row inserted
24968 row inserted
24969 row inserted
24970 row inserted
24971 row inserted
24972 row inserted
24973 row inserted
24974 row inserted
24975 row inserted
24976 row inserted
24977 row inserted
24978 row inserted
24979 row inserted
24980 row inserted
24981 row inserted
24982 row inserted
24983 row inserted
24984 row inserted
24985 row inserted
24986 row inserted
24987 row inserted
24988 row inserted
24989 row inserted
24990 row inserted
24991 row inserted
24992 row inserted
24993 row inserted
24994 row inserted
24995 row inserted
24996 row inserted
24997 row inserted
24998 row inserted
24999 row inserted
25000 row inserted
25001 row inserted
25002 row inserted
25003 row inserted
25004 row inserted
25005 row inserted
25006 row inserted
25007 row inserted
25008 row inserted
25009 row inserted
25010 row inserted
25011 row inserted
25012 row inserted
25013 row in

In [35]:
#SELECT

sql_query = "SELECT * FROM LIQUID_PARTICLE"
stmt = ibm_db.exec_immediate(conn, sql_query)

data = []
row = ibm_db.fetch_assoc(stmt)
while row:
    data.append(row)
    row = ibm_db.fetch_assoc(stmt)
df = pd.DataFrame(data)

a = df['LOCATION'].unique()
df_concat = []
for location in a:
    latest_location = df[df['LOCATION'] == location].nlargest(1, 'TS_DT')
    df_concat.append(latest_location)
latest_data = pd.concat(df_concat, ignore_index=True)


# IF DATA EXIST INSERT
insert_query = "INSERT INTO LIQUID_PARTICLE_LAST (LOCATION, TS_DT, VALUE_1_0, VALUE_3_0, VALUE_5_0, VALUE_10_0, VALUE_15_0, VALUE_20_0, VALUE_25_0, VALUE_50_0) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

insert = ibm_db.prepare(conn, insert_query)
for index, row in latest_data.iterrows():
    location = row['LOCATION']
    timestamp = row['TS_DT']
    c1 = row['VALUE_1_0']
    c3 = row['VALUE_3_0']
    c5 = row['VALUE_5_0']
    c10 = row['VALUE_10_0']
    c15 = row['VALUE_15_0']
    c20 = row['VALUE_20_0']
    c25 = row['VALUE_25_0']
    c50 = row['VALUE_50_0']

    if ibm_db.execute(insert, (location, timestamp, c1, c3, c5, c10, c15, c20, c25, c50)):
        print(f"Row {index + 1} inserted successfully")
    else:
        print(f"Error inserting row {index + 1}: {ibm_db.stmt_errormsg()}")
print('All Rows are inserted')
ibm_db.commit(conn)

Row 1 inserted successfully
Row 2 inserted successfully
Row 3 inserted successfully
Row 4 inserted successfully
Row 5 inserted successfully
Row 6 inserted successfully
Row 7 inserted successfully
Row 8 inserted successfully
Row 9 inserted successfully
Row 10 inserted successfully
Row 11 inserted successfully
Row 12 inserted successfully
Row 13 inserted successfully
All Rows are inserted


True

In [38]:
latest_data.columns

Index(['LOCATION', 'TS_DT', 'VALUE_1_0', 'VALUE_3_0', 'VALUE_5_0',
       'VALUE_10_0', 'VALUE_15_0', 'VALUE_20_0', 'VALUE_25_0', 'VALUE_50_0'],
      dtype='object')

In [39]:
#SELECT

sql_query = "SELECT * FROM LIQUID_PARTICLE"
stmt = ibm_db.exec_immediate(conn, sql_query)

data = []
row = ibm_db.fetch_assoc(stmt)
while row:
    data.append(row)
    row = ibm_db.fetch_assoc(stmt)
df = pd.DataFrame(data)

a = df['LOCATION'].unique()
df_concat = []
for location in a:
    latest_location = df[df['LOCATION'] == location].nlargest(1, 'TS_DT')
    df_concat.append(latest_location)
latest_data = pd.concat(df_concat, ignore_index=True)

# IF DATA NOT EXIST UPDATE

update_query = "UPDATE LIQUID_PARTICLE_LAST SET TS_DT = ?, VALUE_1_0 = ?, VALUE_3_0 = ?, VALUE_5_0 = ?, VALUE_10_0 = ?, VALUE_15_0 = ?, VALUE_20_0 = ?, VALUE_25_0 = ?, VALUE_50_0 = ? WHERE LOCATION = ? "

stmt = ibm_db.prepare(conn, update_query)

for index, row in latest_data.iterrows():
    location = row['LOCATION']
    timestamp = row['TS_DT']
    c1 = row['VALUE_1_0']
    c3 = row['VALUE_3_0']
    c5 = row['VALUE_5_0']
    c10 = row['VALUE_10_0']
    c15 = row['VALUE_15_0']
    c20 = row['VALUE_20_0']
    c25 = row['VALUE_25_0']
    c50 = row['VALUE_50_0']
    
    if ibm_db.execute(stmt, (timestamp, c1, c3, c5, c10, c15, c20, c25, c50, location)):
        print(f"Row {index + 1} updated successfully")
    else:
        print(f"Error updating row {index + 1}: {ibm_db.stmt_errormsg()}")
print('All Rows are uploaded successfully')

Row 1 updated successfully
Row 2 updated successfully
Row 3 updated successfully
Row 4 updated successfully
Row 5 updated successfully
Row 6 updated successfully
Row 7 updated successfully
Row 8 updated successfully
Row 9 updated successfully
Row 10 updated successfully
Row 11 updated successfully
Row 12 updated successfully
Row 13 updated successfully
All Rows are uploaded successfully
